## ДЗ_13 - Модели BERT и GPT.


In [1]:
!pip install transformers sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [2]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [3]:
import pandas as pd

In [4]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [6]:
corpus['train'][0:4]

{'id': ['25349', '25360', '25382', '25423'],
 'id_1': ['34420', '34438', '34475', '34494'],
 'id_2': ['34421', '34439', '34476', '34519'],
 'text_1': ['Цены на нефть восстанавливаются',
  '"Гоголь-центр" покажет видеозапись скандального спектакля "Тангейзер"',
  'Агент: РФС вновь задерживает зарплату Фабио Капелло',
  'День Победы в Москве обещает выдаться облачным'],
 'text_2': ['Парламент Словакии поблагодарил народы бывшего СССР за Победу',
  'Кехман запретил «Гоголь-центру» показывать видеозапись «Тангейзера»',
  'СМИ: Агент Фабио Капелло грозится подать в суд на РФС',
  'Любляна отпразднует День Победы вместе с Москвой'],
 'class': ['-1', '-1', '-1', '-1']}

In [7]:
print(corpus['train']['text_1'][1])
print(corpus['train']['text_2'][1])

"Гоголь-центр" покажет видеозапись скандального спектакля "Тангейзер"
Кехман запретил «Гоголь-центру» показывать видеозапись «Тангейзера»


In [8]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

### Используем предобученную модель 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

In [9]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
result_df = []

def paraphrase_check(row_numb):

    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]

    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()

    p = proba[0]
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"

    result_df.append([text1, text2, p[0], p[1], paraphrase])

    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность False', 'вероятность True', 'Результат'])

In [11]:
pd.set_option('display.max_colwidth', None)

In [12]:
paraphrase_check(0)
paraphrase_check(10)
paraphrase_check(20)
paraphrase_check(30)
paraphrase_check(40)
paraphrase_check(50)
paraphrase_check(60)

,текст 1,текст 2,вероятность False,вероятность True,Результат
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего СССР за Победу,0.945699,0.054301,non
1,Пушков: у Обамы не хватило духа лично поздравить наш народ с Победой,Пушков: Обама не нашел в себе духа лично поздравить россиян с Победой,0.027706,0.972294,paraphrase
2,Лидер либдемов Ник Клегг подал в отставку после неудачи на выборах,Лидер британских лейбористов подал в отставку из-за поражения на выборах,0.080856,0.919144,paraphrase
3,"Прокуратура пока не решила, обжаловать ли приговор Васильевой",Защита Васильевой обжалует приговор суда,0.760585,0.239415,non
4,Владимир Путин освободил от должности почти 20 генералов,Путин снял с должностей более 20 руководителей-силовиков,0.027089,0.972911,paraphrase
5,Премьер Словакии прибыл в Москву для участия в мероприятиях 9 мая,Путеводитель по Дню Победы: как провести 9 мая в Москве,0.946648,0.053352,non
6,Коморовский: отсутствие Меркель в Москве 9 мая важно для Польши,Лукашенко назвал надуманной проблему своего отсутствия 9 Мая в Москве,0.945382,0.054618,non


### ВЫВОД: с задачей классификации парафраза модель справляется успешно.